# 04 — Final Data Prep for Power BI

**Author**: Namora Fernando <br>
**Date**: 2025-08-29 <br>

**Purpose**: Produce a clean, Power BI-ready dataset by refining the engineered data from previous notebook **03_feature_engineering_risk_index.ipynb**. Region mapping will be added, and packaging key outputs into a single Excel workbook for dashboarding. <br>

This notebook assumes the following inputs from earlier notebooks:
- `data_intermiediate/cleaned_merged_inflation_data.csv` (from **01_data_cleaning.ipynb**)
- `data_intermediate/eda_country_summary.csv` (from **02_exploratory_analysis.ipynb**)
- `data_intermediate/inflation_data_with_risk_index.csv` (from **03_feature_engineering_risk_index.ipynb**)

**Outputs**:
- `data_final/inflation_risk_final.csv` — main dataset ready for Power BI dashboard (refined from `inflation_data_with_risk_index.csv`)
- `data_final/inflation_risk_final.xlsx` — Excel contains of 3 sheets:
  - **Final**: refined dataset or Power BI from this notebook (same as `inflation_risk_final.csv`)
  - **EDA_Country_Summary**: country-level summary (same as `eda_country_summary.csv`)
  - **Cleaned_Merged**: cleaned merged data (same as `cleaned_merged_inflation_data.csv`)

> Note: If some values remain missing (NaN) even after prior imputations in **03_feature_engineering_risk_index.ipynb**, we intentionally **keep them as-is**. This is decided since Power BI can handle blanks, and this preserves alignment with source data quality.

## 1. Imports & Paths

In [1]:

# Standard libraries
import os
from pathlib import Path
from typing import Dict, List

# Data handling
import pandas as pd
import numpy as np
import xlsxwriter

# File system setup
BASE_DIR = Path('.')
# Since the folder of data is located the same as this notebook:
DATA_INTERMEDIATE = BASE_DIR / 'data_intermediate'
DATA_FINAL = BASE_DIR / 'data_final'

DATA_FINAL.mkdir(parents=True, exist_ok=True)

# Input files (produced by notebooks Steps 01, 02, 03)
FILE_STEP01 = DATA_INTERMEDIATE / 'cleaned_merged_inflation_data.csv'
FILE_STEP02 = DATA_INTERMEDIATE / 'eda_country_summary.csv'
FILE_STEP03 = DATA_INTERMEDIATE / 'inflation_data_with_risk_index.csv'

# Outputs (this notebook)
FILE_FINAL_CSV = DATA_FINAL / 'inflation_risk_final.csv'
FILE_FINAL_XLSX = DATA_FINAL / 'inflation_risk_final.xlsx'

print('Paths are set.')
print('Step 01    ->', FILE_STEP01)
print('Step 02    ->', FILE_STEP02)
print('Step 03    ->', FILE_STEP03)
print('Output 01  ->', FILE_FINAL_CSV)
print('Output 02  ->', FILE_FINAL_XLSX)

Paths are set.
Step 01    -> data_intermediate\cleaned_merged_inflation_data.csv
Step 02    -> data_intermediate\eda_country_summary.csv
Step 03    -> data_intermediate\inflation_data_with_risk_index.csv
Output 01  -> data_final\inflation_risk_final.csv
Output 02  -> data_final\inflation_risk_final.xlsx


## 2. Load Inputs

In [2]:
# Load Step outputs. If a file is missing, raise a helpful error.
def _read_csv_required(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f'Missing required input: {path}')
    return pd.read_csv(path)

df_cleaned_merged = _read_csv_required(FILE_STEP01)
df_eda_summary = _read_csv_required(FILE_STEP02)
df_risk_index = _read_csv_required(FILE_STEP03)

print('Loaded shapes:')
print('  Step01 (Cleaned Merged):', df_cleaned_merged.shape)
print('  Step02 (EDA Summary):', df_eda_summary.shape)
print('  Step03 (With Risk Index):', df_risk_index.shape)

# Showing columns from Step 03 (the backbone for this final data)
pd.DataFrame({'columns': df_risk_index.columns})

Loaded shapes:
  Step01 (Cleaned Merged): (15918, 7)
  Step02 (EDA Summary): (263, 8)
  Step03 (With Risk Index): (15918, 29)


,columns
0,Country Name
1,Country Code
2,Year
3,CPI_AnnualChange
4,GDP_Growth
5,MoneySupply_GDPpct
6,ExchangeRate_LCUperUSD
7,ExchangeRate_ChangePct
8,CPI_RollingVol_3y
9,MoneySupply_ChangePct


After successfully loaded all dataset from previous steps, proceed to the next steps to refine the dataset.

## 3. Add Region Mapping (Deterministic & Reproducible)

We will attach a `Region` column to the final dataset. In this process the regions are determined specifically, this can be changed later. If some countries not belongs to any region we determined previously, will be determined as `"Other"` by default.

In [3]:
country_to_region: Dict[str, str] = {
    'United States': 'North America',
    'Canada': 'North America',
    'Mexico': 'North America',
    'Brazil': 'Latin America & Caribbean',
    'Argentina': 'Latin America & Caribbean',
    'United Kingdom': 'Europe & Central Asia',
    'Germany': 'Europe & Central Asia',
    'France': 'Europe & Central Asia',
    'Italy': 'Europe & Central Asia',
    'Spain': 'Europe & Central Asia',
    'Netherlands': 'Europe & Central Asia',
    'Poland': 'Europe & Central Asia',
    'Russia': 'Europe & Central Asia',
    'Turkey': 'Europe & Central Asia',
    'South Africa': 'Sub-Saharan Africa',
    'Nigeria': 'Sub-Saharan Africa',
    'Kenya': 'Sub-Saharan Africa',
    'Egypt': 'Middle East & North Africa',
    'Saudi Arabia': 'Middle East & North Africa',
    'United Arab Emirates': 'Middle East & North Africa',
    'Israel': 'Middle East & North Africa',
    'China': 'East Asia & Pacific',
    'Japan': 'East Asia & Pacific',
    'South Korea': 'East Asia & Pacific',
    'Indonesia': 'East Asia & Pacific',
    'Malaysia': 'East Asia & Pacific',
    'Thailand': 'East Asia & Pacific',
    'Vietnam': 'East Asia & Pacific',
    'India': 'South Asia',
    'Pakistan': 'South Asia',
    'Bangladesh': 'South Asia',
    'Sri Lanka': 'South Asia',
    'Australia': 'East Asia & Pacific',
    'New Zealand': 'East Asia & Pacific',
}

def attach_region(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Region'] = df['Country Name'].map(country_to_region).fillna("Unknown")
    return df

df_risk_index_with_region = attach_region(df_risk_index)
print('Reguin unique values:', sorted(df_risk_index_with_region['Region'].unique())[:10], '...')

Reguin unique values: ['East Asia & Pacific', 'Europe & Central Asia', 'Latin America & Caribbean', 'Middle East & North Africa', 'North America', 'South Asia', 'Sub-Saharan Africa', 'Unknown'] ...


# 4. Final Column Formatting (Power BI-friendly)

To do:
- Keep column names **human-readable** while preserving semantics from Step 03 (**03_feature_engineering_risk_index.ipynb**)
- Leave **missing values (NaN)** intact to reflect data limitations transparently.
- Adding Risk Category based on `Risk_Score_0_100` column.

> This step is additionally in order the dataset friendly to be use on Power BI, so that will be not needed any further transformation of the data on Power BI.

In [4]:
final_df = df_risk_index_with_region.copy()

# Rename map specified below for columns that will later be seen on dashboard
# Meanwhile for more technical columns will left as original name
rename_map = {
    'CPI_AnnualChange': 'CPI Annual Change (%)',
    'GDP_Growth': 'GDP Annual Growth (%)',
    'MoneySupply_GDPpct': 'Money Supply (% of GDP)',
    'ExchangeRate_LCUperUSD': 'Exchange Rate (LCU per USD)',
    'ExchangeRate_ChangePct': 'Exchange Rate (% Change YoY)',
    'MoneySupply_ChangePct' : 'MoneySupply (% Change YoY)',
    'CPI_RollingVol_3y' : 'CPI Rolling Volatility (3 years)',
    'GDP_Growth_MA_3y' : 'GDP 3 yrs Growth (Moving Average)',
    'CPI_log' : 'CPI Annual Change (%) [log(1+x) transformed]',
    'FX_log' : 'Exchange Rate (LCU per USD) [log(1+x) transformed]',
    'MS_log' : 'Money Supply (% of GDP) [log(1+x) transformed]',
    'Risk_Index_Z': 'Risk Index (Z-score)',
    'Risk_Score_0_100': 'Risk Score'
}

# Apply only for columns that exist
existing_renames = {k: v for k, v in rename_map.items() if k in final_df.columns}
final_df.rename(columns=existing_renames, inplace=True)

# Create a simple categorization of Risk Score based on its quantiles
# The categorization has 3 values: Low, Medium, High
q_low, q_high = final_df['Risk Score'].quantile([0.33, 0.66])

def _category(x):
    if pd.isna(x):
        return np.nan
    if x <= q_low: return 'Low'
    if x <= q_high: return 'Medium'
    return 'High'

final_df['Risk Category'] = final_df['Risk Score'].apply(_category)
print('Risk Category has been created from Risk Score quantiles (0.33/0.66).')

# Reorder columns (put human-facing first)
preferred_order = [
    c for c in [
        'Country', 'Country Code', 'Region', 'Year',
        'CPI Annual Change (%)',
        'GDP Growth (%)',
        'Money Supply (% of GDP)',
        'Exchange Rate (LCU per USD)',
        'Risk Score', 'Risk Category'
    ] if c in final_df.columns
]
# Include any remaining columns at the end
remaining = [c for c in final_df.columns if c not in preferred_order]
final_df = final_df[preferred_order + remaining]

final_df.head(5)

Risk Category has been created from Risk Score quantiles (0.33/0.66).


,Country Code,Region,Year,CPI Annual Change (%),Money Supply (% of GDP),Exchange Rate (LCU per USD),Risk Score,Risk Category,Country Name,GDP Annual Growth (%),...,CPI_log_win,FX_log_win,MS_log_win,GDP_Growth_win,z_CPI,z_FX,z_MS,z_GDP,z_GDP_inv,Risk Index (Z-score)
0,AFG,Unknown,1960,12.686269,NaN,17.196561,73.682224,High,Afghanistan,-9.431974,...,2.616393,0.000000,1.13082,-9.431974,0.665760,-0.033170,0.633485,-2.388371,2.388371,0.742964
1,AFG,Unknown,1961,12.686269,NaN,17.196561,77.382261,High,Afghanistan,-9.431974,...,2.616393,0.000000,1.13082,-9.431974,0.637773,-0.013548,0.662564,-2.331492,2.331492,0.733959
2,AFG,Unknown,1962,12.686269,NaN,17.196561,63.251059,High,Afghanistan,-9.431974,...,2.616393,0.000000,1.13082,-9.431974,0.616062,-0.187294,0.523760,-2.704354,2.704354,0.710006
3,AFG,Unknown,1963,12.686269,NaN,35.109645,88.498185,High,Afghanistan,-9.431974,...,2.616393,4.151438,1.13082,-9.431974,0.596935,4.843493,0.519742,-2.587929,2.587929,1.941785
4,AFG,Unknown,1964,12.686269,NaN,38.692262,79.236123,High,Afghanistan,-9.431974,...,2.616393,2.416278,1.13082,-9.431974,0.587028,2.343264,0.672469,-2.803802,2.803802,1.375691


## 5. Data Quality Note — Missing Values

Some indicators may still contain missing values (NaN) despite imputations in Step 03. This reflects genuine data gaps in source series. We intentionally **leave NaN as blanks** to preserve transparency and let Power BI handle them during visual aggregation or filtering.

In [5]:
# Lightweight missingness snapshot
missing_summary = final_df.isna().mean().sort_values(ascending=False)
print('Share of missing values per column (0–1):')
missing_summary.head(20)

Share of missing values per column (0–1):


Risk Score                                            0.383717
Risk Category                                         0.383717
Risk Index (Z-score)                                  0.383717
Money Supply (% of GDP)                               0.319764
CPI Rolling Volatility (3 years)                      0.305252
Exchange Rate (LCU per USD)                           0.226787
Exchange Rate (% Change YoY)                          0.195628
Exchange Rate (LCU per USD) [log(1+x) transformed]    0.195628
z_FX                                                  0.195628
FX_log_win                                            0.195628
MS_log_win                                            0.189848
z_MS                                                  0.189848
Money Supply (% of GDP) [log(1+x) transformed]        0.189848
MoneySupply (% Change YoY)                            0.189848
GDP 3 yrs Growth (Moving Average)                     0.110692
z_CPI                                                 0

## 6. Save Final Outputs

In [6]:
# Save final CSV (Power BI–ready backbone)
final_df.to_csv(FILE_FINAL_CSV, index=False)
print('Saved:', FILE_FINAL_CSV)

# Save Excel workbook with 3 sheets:
#   1. Final (this refined dataset)
#   2. EDA_Country_Summary (from Step 02)
#   3. Cleaned_Merged (from Step 01)
with pd.ExcelWriter(FILE_FINAL_XLSX, engine='xlsxwriter') as writer:
    final_df.to_excel(writer, sheet_name='Final', index=False)
    df_eda_summary.to_excel(writer, sheet_name='EDA_Country_Summary', index=False)
    df_cleaned_merged.to_excel(writer, sheet_name='Cleaned_Merged', index=False)

print('Saved:', FILE_FINAL_XLSX)

Saved: data_final\inflation_risk_final.csv
Saved: data_final\inflation_risk_final.xlsx


## 7. Quick QA Checks

Below just to ensure our progress on this notebook has been done successfully as we performed throughout notebook.

In [7]:
# Basic sanity checks
assert final_df.shape[0] > 0, 'Final dataset is empty.'

# Check key columns exist
for col in ['Country Name', 'Year', 'Risk Score']:
    if col not in final_df.columns:
        print(f'Warning: expected column missing -> {col}')

# Spot-check year bounds
print('Year min/max:', final_df['Year'].min(), final_df['Year'].max())

# Show sample rows
display(final_df.sample(min(5, len(final_df))))

print('QA checks completed.')

Year min/max: 1960 2024


,Country Code,Region,Year,CPI Annual Change (%),Money Supply (% of GDP),Exchange Rate (LCU per USD),Risk Score,Risk Category,Country Name,GDP Annual Growth (%),...,CPI_log_win,FX_log_win,MS_log_win,GDP_Growth_win,z_CPI,z_FX,z_MS,z_GDP,z_GDP_inv,Risk Index (Z-score)
3132,CIV,Unknown,2003,3.296807,13.109122,579.897426,55.458410,Medium,Cote d'Ivoire,-4.725867,...,1.457872,-4.605170,-4.605170,-4.725867,0.113131,-0.942747,-1.364326,-2.133072,2.133072,-0.143339
361,ASM,Unknown,2018,NaN,NaN,1.000000,NaN,NaN,American Samoa,2.671119,...,NaN,0.000000,NaN,2.671119,NaN,0.434307,NaN,-0.166102,0.166102,NaN
799,AUS,East Asia & Pacific,2003,2.732596,74.595328,1.541914,58.170438,Medium,Australia,3.090762,...,1.317104,-4.605170,1.975702,3.090762,0.021348,-0.942747,0.800181,-0.232279,0.232279,-0.032270
7786,KWT,Unknown,2001,1.300000,86.060952,0.306682,48.302491,Medium,Kuwait,0.213333,...,0.832909,-0.023056,3.134645,0.213333,-0.458051,-0.761070,0.743530,-0.772736,0.772736,-0.108872
14781,TUR,Unknown,1989,63.272552,26.815438,0.002122,60.542564,High,Turkiye,0.290244,...,4.163133,3.915369,-4.605170,0.290244,1.150351,0.937488,-1.256652,-0.477075,0.477075,0.514743


QA checks completed.


## 8. Summary & Next Steps

This notebook successfully done a few things:
  - Added Region mapping
  - Friendly names of columns to be presented
  - Excel packaging

**Next Steps**: <br>
Build Power BI dashboards using `data_final/inflation_risk_final.xlsx` (sheet **Final** as the primary model table). Optionally join `EDA_Country_Summary` or `Cleaned_Merged` for validation views.